In [1]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import os

In [2]:
def get_last_trading_day(df):
    # Ensure 'Timestamp' is in datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    # Get the last trading day
    last_day = df['Timestamp'].max()
    # Adjust for weekends (Saturday=5, Sunday=6)
    while last_day.weekday() >= 5:  # Saturday or Sunday
        last_day -= pd.Timedelta(days=1)
    return last_day

In [3]:
def trade_simulation(num_days, minute_file, fifteen_minute_file):
    all_results = []
    monthly_results = {}

    try:
        Odata_15m = pd.read_excel(fifteen_minute_file, index_col=None).drop(columns=['Unnamed: 0'], errors='ignore')
        Odata_1m = pd.read_excel(minute_file, index_col=None).drop(columns=['Unnamed: 0'], errors='ignore')
    except Exception as e:
        print(f"Error loading data: {e}")
        return
    

    end_date = get_last_trading_day(Odata_15m)
    start_date = end_date - pd.Timedelta(days=num_days + 14)
    
    Odata_15m['Timestamp'] = pd.to_datetime(Odata_15m['Timestamp'])
    Odata_1m['Timestamp'] = pd.to_datetime(Odata_1m['Timestamp'])

    trading_days = [end_date - pd.Timedelta(days=i) for i in range(num_days)]

    for trading_day in trading_days:
        trading_day_str = trading_day.strftime('%Y-%m-%d')
        prev_14_days_end = trading_day - pd.Timedelta(days=1)
        prev_14_days_start = prev_14_days_end - pd.Timedelta(days=14)
        next_trading_day = trading_day + pd.Timedelta(days=1)
        next_trading_day_str = next_trading_day.strftime('%Y-%m-%d')

        try:
            data_15m = Odata_15m[(Odata_15m['Timestamp'] >= prev_14_days_start) & (Odata_15m['Timestamp'] <= prev_14_days_end)]
            data_1m = Odata_1m[(Odata_1m['Timestamp'] >= trading_day_str) & (Odata_1m['Timestamp'] < next_trading_day_str)]

            if data_15m.empty or data_1m.empty:
                continue

            supports = data_15m[data_15m.Low == data_15m.Low.rolling(5, center=True).min()]['Low']
            resistances = data_15m[data_15m.High == data_15m.High.rolling(5, center=True).max()]['High']

            supports = supports.reindex(data_15m.index)
            resistances = resistances.reindex(data_15m.index)

            def near_divisible_by_500(level):
                return abs(level % 100) <= 15 or abs(level % 100 - 100) <= 15

            def count_touches(level, data):
                return ((level >= data['Open']) & (level <= data['Close'])).sum()

            filtered_supports = supports[
                (supports < data_15m['Open']) & (supports < data_15m['Close']) & supports.apply(near_divisible_by_500)
            ]
            filtered_resistances = resistances[
                (resistances > data_15m['Open']) & (resistances > data_15m['Close']) & resistances.apply(near_divisible_by_500)
            ]

            filtered_supports = filtered_supports[filtered_supports.apply(lambda x: 2 < count_touches(x, data_15m) <= 5)]
            filtered_resistances = filtered_resistances[filtered_resistances.apply(lambda x: 2 < count_touches(x, data_15m) <= 5)]

            levels = pd.concat([filtered_supports, filtered_resistances]).sort_index()

            trade_results = []
            trade_executed = False

            # Get the closing price at 3:15 PM
            closing_price = data_1m[data_1m['Timestamp'].dt.time == pd.to_datetime('15:15:00').time()]['Close'].iloc[-1] if not data_1m[data_1m['Timestamp'].dt.time == pd.to_datetime('15:15:00').time()].empty else None

            for i, row in data_1m.iterrows():
                if trade_executed:
                    break
                for level in levels:
                    if row['Low'] <= level <= row['High']:
                        open_price = row['Open']
                        trade_type, stoploss, target = ('Buy', level - 100, level + 210) if open_price > level else ('Sell', level + 100, level - 210)
                        
                        hit_target = hit_stoploss = False
                        entry_candle = row['Timestamp']
                        exit_candle = None
                        for j, next_row in data_1m.loc[i:].iterrows():
                            if trade_type == 'Buy':
                                if next_row['Low'] <= stoploss:
                                    hit_stoploss = True
                                    exit_candle = next_row['Timestamp']
                                    break
                                if next_row['High'] >= target:
                                    hit_target = True
                                    exit_candle = next_row['Timestamp']
                                    break
                            else:
                                if next_row['High'] >= stoploss:
                                    hit_stoploss = True
                                    exit_candle = next_row['Timestamp']
                                    break
                                if next_row['Low'] <= target:
                                    hit_target = True
                                    exit_candle = next_row['Timestamp']
                                    break

                        points = (target - level) if trade_type == 'Buy' and hit_target else (stoploss - level) if trade_type == 'Buy' and hit_stoploss else (level - target) if trade_type == 'Sell' and hit_target else (level - stoploss) if trade_type == 'Sell' and hit_stoploss else 0
                        
                        result = {
                            'Trading Day': trading_day_str,
                            'Entry Candle': entry_candle,
                            'Exit Candle': exit_candle,
                            'Level': level,
                            'Trade Type': trade_type,
                            'Stoploss': stoploss,
                            'Target': target,
                            'Hit Target': hit_target,
                            'Hit Stoploss': hit_stoploss,
                            'Result': 'Loss' if hit_stoploss else ('Win' if hit_target else 'Loss'),
                            'Points': points,
                            'Closing Price at 3:15 PM': closing_price
                        }
                        trade_results.append(result)
                        trade_executed = True
                        break  # Stop checking further levels once a trade is executed

            all_results.extend(trade_results)

                        # Collect monthly results
            for result in trade_results:
                month = pd.to_datetime(result['Trading Day']).strftime('%Y-%m')
                if month not in monthly_results:
                    monthly_results[month] = []
                monthly_results[month].append(result)

        except Exception as e:
            print(f"Error processing {trading_day_str}: {e}")
            continue

    file_path = 'results.xlsx'
    try:
        if os.path.exists(file_path):
            workbook = load_workbook(file_path)
            sheet = workbook.get_sheet_by_name('Results') if 'Results' in workbook.sheetnames else workbook.create_sheet(title='Results')
        else:
            workbook = Workbook()
            sheet = workbook.active
            sheet.title = 'Results'

        df_all_results = pd.DataFrame(all_results)
        total_wins = df_all_results['Result'].value_counts().get('Win', 0)
        total_losses = df_all_results['Result'].value_counts().get('Loss', 0)
        total_gain_points = df_all_results[df_all_results['Result'] == 'Win']['Points'].sum()
        total_loss_points = df_all_results[df_all_results['Result'] == 'Loss']['Points'].sum()
        net_total_points = total_gain_points + total_loss_points

        df_all_results['Entry Candle'] = df_all_results['Entry Candle'].dt.tz_localize(None)
        df_all_results['Exit Candle'] = df_all_results['Exit Candle'].dt.tz_localize(None)

        for r_idx, row in enumerate(dataframe_to_rows(df_all_results, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                sheet.cell(row=r_idx, column=c_idx, value=value)

        summary_stats = [
            ['Total Wins', total_wins],
            ['Total Losses', total_losses],
            ['Total Gain Points', total_gain_points],
            ['Total Loss Points', total_loss_points],
            ['Net Total Points', net_total_points]
        ]
        summary_start_row = r_idx + 2

        for i, (stat, value) in enumerate(summary_stats):
            sheet.cell(row=summary_start_row + i, column=1, value=stat)
            sheet.cell(row=summary_start_row + i, column=2, value=value)
        
                # Add monthly sheets
        for month, results in monthly_results.items():
            if month not in workbook.sheetnames:
                monthly_sheet = workbook.create_sheet(title=month)
            else:
                monthly_sheet = workbook[month]

            df_monthly_results = pd.DataFrame(results)
            for r_idx, row in enumerate(dataframe_to_rows(df_monthly_results, index=False, header=True), 1):
                for c_idx, value in enumerate(row, 1):
                    monthly_sheet.cell(row=r_idx, column=c_idx, value=value)
                    
            # Calculate monthly summary stats
            total_wins_month = df_monthly_results['Result'].value_counts().get('Win', 0)
            total_losses_month = df_monthly_results['Result'].value_counts().get('Loss', 0)
            total_gain_points_month = df_monthly_results[df_monthly_results['Result'] == 'Win']['Points'].sum()
            total_loss_points_month = df_monthly_results[df_monthly_results['Result'] == 'Loss']['Points'].sum()
            net_total_points_month = total_gain_points_month + total_loss_points_month

            monthly_summary_stats = [
                ['Total Wins', total_wins_month],
                ['Total Losses', total_losses_month],
                ['Total Gain Points', total_gain_points_month],
                ['Total Loss Points', total_loss_points_month],
                ['Net Total Points', net_total_points_month]
            ]
            monthly_summary_start_row = len(df_monthly_results) + 3

            for i, (stat, value) in enumerate(monthly_summary_stats):
                monthly_sheet.cell(row=monthly_summary_start_row + i, column=1, value=stat)
                monthly_sheet.cell(row=monthly_summary_start_row + i, column=2, value=value)

        workbook.save(file_path)

    except Exception as e:
        print(f"Error saving results to Excel: {e}")

    return all_results

trade_simulation(num_days=180, minute_file='1_min.xlsx', fifteen_minute_file='15_min.xlsx')


[{'Trading Day': '2024-08-23',
  'Entry Candle': Timestamp('2024-08-23 09:15:00'),
  'Exit Candle': None,
  'Level': 50910.9,
  'Trade Type': 'Buy',
  'Stoploss': 50810.9,
  'Target': 51120.9,
  'Hit Target': False,
  'Hit Stoploss': False,
  'Result': 'Loss',
  'Points': 0,
  'Closing Price at 3:15 PM': 50920.15},
 {'Trading Day': '2024-08-22',
  'Entry Candle': Timestamp('2024-08-22 09:15:00'),
  'Exit Candle': Timestamp('2024-08-22 09:37:00'),
  'Level': 50908.65,
  'Trade Type': 'Sell',
  'Stoploss': 51008.65,
  'Target': 50698.65,
  'Hit Target': False,
  'Hit Stoploss': True,
  'Result': 'Loss',
  'Points': -100.0,
  'Closing Price at 3:15 PM': 51001.0},
 {'Trading Day': '2024-08-21',
  'Entry Candle': Timestamp('2024-08-21 09:15:00'),
  'Exit Candle': Timestamp('2024-08-21 09:32:00'),
  'Level': 50707.75,
  'Trade Type': 'Sell',
  'Stoploss': 50807.75,
  'Target': 50497.75,
  'Hit Target': True,
  'Hit Stoploss': False,
  'Result': 'Win',
  'Points': 210.0,
  'Closing Price at 3